# prepare funcitons

In [12]:
def get_ntype_range(target_ntype):
    start_idx = 0
    for ntype in g[0].ntypes:
        if ntype == target_ntype:
            end_idx = start_idx + g[0].num_nodes(ntype)
            return start_idx, end_idx
        start_idx += g[0].num_nodes(ntype)

In [13]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as Metric
from sklearn.metrics import f1_score
import torch as th

def concat_u_v(x, u_idx, v_idx):
    u = x[u_idx]
    v = x[v_idx]
    emd = th.cat((u, v), dim=1)
    return emd

def f1_node_classification(y_label, y_pred):
    macro_f1 = f1_score(y_label, y_pred, average='macro')
    micro_f1 = f1_score(y_label, y_pred, average='micro')
    return macro_f1, micro_f1

# Used in HetGNN
def LR_pred(train_X, train_Y, test_X):
    LR = LogisticRegression(max_iter=10000, random_state=0)
    LR.fit(train_X, train_Y)
    pred_Y = LR.predict(test_X)
    # AUC_score = Metric.roc_auc_score(test_Y, pred_Y)
    return pred_Y

def link_prediction( train_X, train_Y, test_X, test_Y):
    pred_Y = LR_pred(train_X, train_Y, test_X)
    AUC_score = Metric.roc_auc_score(test_Y, pred_Y)
    macro_f1, micro_f1 = f1_node_classification(test_Y, pred_Y)
    return AUC_score, macro_f1, micro_f1

def author_link_prediction(x, train_batch, test_batch):
    train_u, train_v, train_Y = train_batch
    test_u, test_v, test_Y = test_batch

    train_X = concat_u_v(x, th.tensor(train_u), th.tensor(train_v))
    test_X = concat_u_v(x, th.tensor(test_u), th.tensor(test_v))
    train_Y = th.tensor(train_Y)
    test_Y = th.tensor(test_Y)
    return link_prediction(train_X, train_Y, test_X, test_Y)

# two year-split

In [14]:
def load_link_pred(path):
    u_list = []
    v_list = []
    label_list = []
    with open(path) as f:
        for i in f.readlines( ):
            u, v, label = i.strip( ).split(', ')
            u_list.append(int(u))
            v_list.append(int(v))
            label_list.append(int(label))
    return u_list, v_list, label_list
train_batch = load_link_pred('/data/jx4237data/TKG/new_TKG/TKG_new_BAI/630deliverable/OpenHGNN/openhgnn/dataset/academic4HetGNN/a_a_list_train_2015_modified.txt')
test_batch = load_link_pred('/data/jx4237data/TKG/new_TKG/TKG_new_BAI/630deliverable/OpenHGNN/openhgnn/dataset/academic4HetGNN/a_a_list_test_2015_modified.txt')

# agg

In [15]:
import pickle
with open('/data/jx4237data/TKG/new_TKG/TKG_new_BAI/630deliverable/test/author_embeddings.pkl', 'rb') as f:
    author_embeddings = pickle.load(f)

In [16]:
import torch
from sklearn.decomposition import PCA

# Convert the tensor to a NumPy array
author_embeddings_np = author_embeddings.numpy()

# Apply PCA
pca = PCA(n_components=5)
reduced_embeddings_np = pca.fit_transform(author_embeddings_np)

# Convert back to a PyTorch tensor
reduced_embeddings = torch.tensor(reduced_embeddings_np, dtype=torch.float32)
author_link_prediction(reduced_embeddings, train_batch, test_batch)

(0.532143197162052, 0.5321131722163304, 0.5321431971620519)

# metapath

In [17]:
import numpy as np

# Load the .npy file
embeddings = np.load('academic4HetGNN_APVPA_mp2vec_embeddings.npy')
from dgl.data.utils import load_graphs
g, _ = load_graphs('/data/jx4237data/TKG/new_TKG/TKG_new_BAI/630deliverable/OpenHGNN/openhgnn/dataset/academic4HetGNN/graph.bin')
start_idx, end_idx = get_ntype_range('author')

In [18]:
metaebd = th.tensor(embeddings[start_idx:end_idx])

In [19]:
metaebd.shape

torch.Size([35751, 128])

In [20]:
import torch
from sklearn.decomposition import PCA

# Convert the tensor to a NumPy array
metaebd_np = metaebd.numpy()

# Apply PCA
pca = PCA(n_components=128)
reduced_embeddings_np = pca.fit_transform(metaebd_np)

# Convert back to a PyTorch tensor
reduced_embeddings = torch.tensor(reduced_embeddings_np, dtype=torch.float32)
author_link_prediction(reduced_embeddings, train_batch, test_batch)

(0.5638765972771262, 0.5607311170696339, 0.5638765972771262)

# hetgnn

In [21]:
import torch
logits = torch.load('/data/jx4237data/TKG/new_TKG/TKG_new_BAI/630deliverable/OpenHGNN/openhgnn/output/HetGNN/embeddings.pt')

In [22]:
import torch
from sklearn.decomposition import PCA

# Convert the tensor to a NumPy array
logits_np = logits.numpy()

# Apply PCA
pca = PCA(n_components=5)
reduced_embeddings_np = pca.fit_transform(logits_np)

# Convert back to a PyTorch tensor
reduced_embeddings = torch.tensor(reduced_embeddings_np, dtype=torch.float32)
author_link_prediction(reduced_embeddings, train_batch, test_batch)

(0.5370473087924234, 0.5360103875918922, 0.5370473087924234)

# hgt

In [23]:
import torch
HGT_logits = torch.load('/data/jx4237data/TKG/new_TKG/TKG_new_BAI/630deliverable/OpenHGNN/openhgnn/output/HGT/HGT_embeddings.pt',map_location=torch.device('cpu'))

In [24]:
HGT_logits['author'].shape
import pickle
new_tensor2oldtensor = pickle.load(open('/data/jx4237data/TKG/new_TKG/TKG_new_BAI/630deliverable/new_tensor2oldtensor.pkl','rb'))

In [25]:
import torch

original_tensor = HGT_logits['author']

# Ensure the original tensor is on the same device as the new row tensor
device = original_tensor.device  # Get the device of the original tensor

# Create a new row tensor with the same number of columns (4) on the same device
new_row = torch.zeros((1, 4), device=device)

# Concatenate the new row to the original tensor
updated_tensor = torch.cat((original_tensor, new_row), dim=0)

# Assuming new_tensor2oldtensor is a dictionary mapping new indices to old indices
# Ensure the index tensor is created on the same device
index_tensor = torch.tensor([new_tensor2oldtensor[i] for i in range(35751)], device=device)

# Reorder the updated tensor based on the index tensor
reordered_tensor = updated_tensor[index_tensor]

# Check the shape of the reordered tensor
print(f"Reordered tensor shape: {reordered_tensor.shape}")

# Update the HGT_logits dictionary with the reordered tensor
HGT_logits['author'] = reordered_tensor

Reordered tensor shape: torch.Size([35751, 4])


In [26]:
HGTLOGITS = HGT_logits['author'].cpu()  # Move the tensor to the CPU

# Convert the tensor to a NumPy array
HGTLOGITS_np = HGTLOGITS.numpy()

# Apply PCA
pca = PCA(n_components=4)
reduced_embeddings_np = pca.fit_transform(HGTLOGITS_np)

# Convert back to a PyTorch tensor
reduced_embeddings = torch.tensor(reduced_embeddings_np, dtype=torch.float32)
author_link_prediction(reduced_embeddings, train_batch, test_batch)

(0.502896520036592, 0.49331052608409853, 0.502896520036592)